In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score, auc
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.model_selection import StratifiedKFold
import category_encoders
from catboost import Pool
from catboost.utils import get_roc_curve
from sklearn.metrics import auc
import pickle
import catboost

In [2]:
train_data = pd.read_csv('./dataset/train_public2.csv')
test_public = pd.read_csv('./dataset/test_public2.csv')
train_internet = pd.read_csv('./dataset/train_internet.csv')

In [3]:
temp = pd.read_csv('./dataset/train_public2.csv')
temp['earlies_credit_mon'] = pd.to_datetime(temp['earlies_credit_mon'])
def f(x):
    #print(x)
    if x>= pd.to_datetime('2021-01-01'):
        t = '20' + str(x)[8:10] + '-' + str(x)[5:7] + '-01'
        #print('t=', t)
        return pd.to_datetime(t)
    return x
temp['earlies_credit_mon'] = temp['earlies_credit_mon'].apply(f)
temp.to_pickle('./dataset/train_public.pkl')

In [4]:
temp = pd.read_csv('./dataset/test_public2.csv')
temp['earlies_credit_mon'] = pd.to_datetime(temp['earlies_credit_mon'])
def f(x):
    #print(x)
    if x>= pd.to_datetime('2021-01-01'):
        t = '20' + str(x)[8:10] + '-' + str(x)[5:7] + '-01'
        #print('t=', t)
        return pd.to_datetime(t)
    return x
temp['earlies_credit_mon'] = temp['earlies_credit_mon'].apply(f)
temp.to_pickle('./dataset/test_public.pkl')

In [5]:
import warnings
warnings.filterwarnings('ignore')

# 数据加载
train1 = pd.read_csv('./dataset/train_internet.csv')
with open('./dataset/train_public.pkl', 'rb') as file:
    train2 = pickle.load(file)
with open('./dataset/test_public.pkl', 'rb') as file:
    test = pickle.load(file)
    
train2

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,...,policy_code,f0,f1,f2,f3,f4,early_return,early_return_amount,early_return_amount_3mon,isDefault
0,1040418,240418,31818.18182,3,11.466,1174.91,C,政府机构,金融业,3 years,...,1,1.0,0.0,4.0,5.0,4.0,3,9927,0.0,0
1,1025197,225197,28000.00000,5,16.841,670.69,C,政府机构,金融业,10+ years,...,1,7.0,0.0,4.0,45.0,22.0,0,0,0.0,0
2,1009360,209360,17272.72727,3,8.900,603.32,A,政府机构,公共服务、社会组织,10+ years,...,1,6.0,0.0,6.0,28.0,19.0,0,0,0.0,0
3,1039708,239708,20000.00000,3,4.788,602.30,A,世界五百强,文化和体育业,6 years,...,1,5.0,0.0,10.0,15.0,9.0,0,0,0.0,0
4,1027483,227483,15272.72727,3,12.790,470.31,C,政府机构,信息传输、软件和信息技术服务业,< 1 year,...,1,10.0,0.0,6.0,15.0,4.0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1028093,228093,17727.27273,3,15.037,510.27,B,普通企业,建筑业,7 years,...,1,4.0,0.0,4.0,11.0,7.0,2,5287,0.0,0
9996,1043911,243911,13636.36364,3,6.534,464.95,A,政府机构,农、林、牧、渔业,2 years,...,1,2.0,0.0,2.0,7.0,6.0,3,7182,0.0,0
9997,1023503,223503,24818.18182,3,14.421,708.69,B,普通企业,信息传输、软件和信息技术服务业,10+ years,...,1,6.0,0.0,5.0,15.0,11.0,1,8540,2562.0,0
9998,1024616,224616,20000.00000,3,18.450,727.58,D,政府机构,农、林、牧、渔业,10+ years,...,1,7.0,0.0,5.0,17.0,10.0,2,6161,616.1,0


In [6]:
import datetime

train2['issue_date'] = pd.to_datetime(train2['issue_date'])
# 提取时间diff
# 设置初始的时间
base_time = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
# 转换为天为单位
train2['issue_date_diff'] = train2['issue_date'].apply(lambda x: x-base_time).dt.days

In [7]:
train2['earlies_credit_mon'] = pd.to_datetime(train2['earlies_credit_mon'])
# 最小日期为 1952-06-01, 最大日期为 2015-05-01
# 设置初始的时间
base_time = datetime.datetime.strptime('1952-06-01', '%Y-%m-%d')
# 转换为天为单位
train2['earlies_credit_mon_diff'] = train2['earlies_credit_mon'].apply(lambda x: x-base_time).dt.days
train2['issue_earlies_diff'] = (train2['issue_date'] - train2['earlies_credit_mon']).dt.days
train2[['earlies_credit_mon', 'earlies_credit_mon_diff', 'issue_earlies_diff']]

,earlies_credit_mon,earlies_credit_mon_diff,issue_earlies_diff
0,2001-12-01,18080,5418
1,1990-04-01,13818,8462
2,1991-10-01,14366,8128
3,2001-06-01,17897,5143
4,2002-05-01,18231,5175
...,...,...,...
9995,2006-02-01,19603,2830
9996,1997-05-01,16405,6788
9997,1987-02-01,12663,9435
9998,1992-10-01,14732,9282


In [8]:
test['issue_date'] = pd.to_datetime(test['issue_date'])
test['earlies_credit_mon'] = pd.to_datetime(test['earlies_credit_mon'])
# 最小日期为 1959-06-01, 最大日期为 2014-11-01
# 设置初始的时间
base_time = datetime.datetime.strptime('1952-06-01', '%Y-%m-%d')
# 转换为天为单位
test['earlies_credit_mon_diff'] = test['earlies_credit_mon'].apply(lambda x: x-base_time).dt.days
test['issue_earlies_diff'] = (test['issue_date'] - test['earlies_credit_mon']).dt.days
test[['earlies_credit_mon', 'earlies_credit_mon_diff', 'issue_earlies_diff']]

,earlies_credit_mon,earlies_credit_mon_diff,issue_earlies_diff
0,2003-03-01,18535,5389
1,1999-12-01,17349,5691
2,1999-04-01,17105,5114
3,2000-07-01,17562,6362
4,2007-05-01,20057,3319
...,...,...,...
4995,2005-07-01,19388,2010
4996,2012-04-01,21854,1856
4997,1986-10-01,12540,9923
4998,1999-03-01,17074,4933


In [9]:
train2.drop('issue_date', axis = 1, inplace = True)
train2

employer_type = train1['employer_type'].value_counts().index
industry = train1['industry'].value_counts().index
# 标签编码
emp_type_dict = dict(zip(employer_type, [0,1,2,3,4,5]))
industry_dict = dict(zip(industry, [i for i in range(15)]))

# work_year缺失值较多，用 10+ years填充
train2['work_year'].fillna('10+ years', inplace=True)
# 标签编码
work_year_map = {'10+ years': 10, '2 years': 2, '< 1 year': 0, '3 years': 3, '1 year': 1,
     '5 years': 5, '4 years': 4, '6 years': 6, '8 years': 8, '7 years': 7, '9 years': 9}
train2['work_year']  = train2['work_year'].map(work_year_map)

train2['class'] = train2['class'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6})
train2['employer_type'] = train2['employer_type'].map(emp_type_dict)
train2['industry'] = train2['industry'].map(industry_dict)


In [10]:
# 日期类型：issueDate，earliesCreditLine
#train[cat_features]
# 转换为pandas中的日期类型
test['issue_date'] = pd.to_datetime(test['issue_date'])
# 提取时间diff
# 设置初始的时间
base_time = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
# 转换为天为单位
test['issue_date_diff'] = test['issue_date'].apply(lambda x: x-base_time).dt.days
test.drop('issue_date', axis = 1, inplace = True)
test['work_year'].fillna('10+ years', inplace=True)

work_year_map = {'10+ years': 10, '2 years': 2, '< 1 year': 0, '3 years': 3, '1 year': 1,
     '5 years': 5, '4 years': 4, '6 years': 6, '8 years': 8, '7 years': 7, '9 years': 9}
test['work_year']  = test['work_year'].map(work_year_map)
test['class'] = test['class'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6})
test['employer_type'] = test['employer_type'].map(emp_type_dict)
test['industry'] = test['industry'].map(industry_dict)

In [11]:
train2['pro'] = train2['interest'].mul(train2['year_of_loan'])

train2['loan_year'] = train2['total_loan']/train2['year_of_loan']
train2['early_ratio'] = train2['early_return_amount']/train2['total_loan']
train2['early_times_ratio'] = train2['early_return']/train2['year_of_loan']
train2['recircle_ratio'] = train2['recircle_b']/train2['total_loan']

train2['recircle_amt'] = train2['recircle_b'].mul(train2['total_loan'])

In [12]:
test['pro'] = test['interest'].mul(test['year_of_loan'])

test['loan_year'] = test['total_loan']/test['year_of_loan']
test['early_ratio'] = test['early_return_amount']/test['total_loan']
test['early_times_ratio'] = test['early_return']/test['year_of_loan']
test['recircle_ratio'] = test['recircle_b']/test['total_loan']

test['recircle_amt'] = test['recircle_b'].mul(test['total_loan'])

In [13]:
train1.to_pickle('./train1.pkl') # 对应train_interest
train2.to_pickle('./train2.pkl') # 对应train_public
test.to_pickle('./test.pkl') # 对应test_public

# 模型

In [14]:
with open('./train2.pkl', 'rb') as file:
    train2 = pickle.load(file)
with open('./test.pkl', 'rb') as file:
    test = pickle.load(file)
# 去掉policy_code字段
train2.drop(['policy_code'], axis=1,inplace=True)
test.drop(['policy_code'], axis=1,inplace=True)

In [15]:
X_train2 = train2.drop(['isDefault','loan_id','user_id', 'app_type', 'pub_dero_bankrup', 'f1', 'del_in_18month', 'initial_list_status'], axis = 1, inplace = False)
y_train2 = train2['isDefault']
X_test = test[X_train2.columns]

In [16]:
cat_model = catboost.CatBoostClassifier(iterations=12000, 
                              depth=7, 
                              learning_rate=0.005, 
                              loss_function='Logloss',
                              eval_metric='AUC',
                              logging_level='Verbose', 
                              metric_period=500)
cat_model.fit(X_train2, y_train2, eval_set=(X_train2, y_train2))

0:	test: 0.8551197	best: 0.8551197 (0)	total: 158ms	remaining: 31m 33s
500:	test: 0.9076838	best: 0.9076838 (500)	total: 3.9s	remaining: 1m 29s
1000:	test: 0.9261204	best: 0.9261204 (1000)	total: 7.62s	remaining: 1m 23s
1500:	test: 0.9418552	best: 0.9418552 (1500)	total: 11.4s	remaining: 1m 19s
2000:	test: 0.9554894	best: 0.9554894 (2000)	total: 15.1s	remaining: 1m 15s
2500:	test: 0.9671795	best: 0.9671795 (2500)	total: 19s	remaining: 1m 12s
3000:	test: 0.9762336	best: 0.9762336 (3000)	total: 22.8s	remaining: 1m 8s
3500:	test: 0.9833034	best: 0.9833034 (3500)	total: 26.5s	remaining: 1m 4s
4000:	test: 0.9887103	best: 0.9887103 (4000)	total: 30.2s	remaining: 1m
4500:	test: 0.9929368	best: 0.9929368 (4500)	total: 34s	remaining: 56.7s
5000:	test: 0.9955501	best: 0.9955501 (5000)	total: 37.8s	remaining: 52.8s
5500:	test: 0.9973815	best: 0.9973815 (5500)	total: 41.5s	remaining: 49s
6000:	test: 0.9984835	best: 0.9984835 (6000)	total: 45.2s	remaining: 45.2s
6500:	test: 0.9991200	best: 0.999120

### 重要性探索 

In [22]:
feature_importances = cat_model.get_feature_importance(prettified=True,
                                                       thread_count=-1,
                                                       verbose=False)

In [25]:
for i in range(feature_importances['Feature Id'].size):
    if feature_importances['Importances'][i]>1:
        X_train2[feature_importances['Feature Id'][i]+'_cnt'] = category_encoders.count.CountEncoder().fit_transform(X_train2[feature_importances['Feature Id'][i]])
        X_test[feature_importances['Feature Id'][i]+'_cnt'] = category_encoders.count.CountEncoder().fit_transform(X_test[feature_importances['Feature Id'][i]])

In [29]:
cat_model = catboost.CatBoostClassifier(iterations=12000, 
                              depth=7, 
                              learning_rate=0.005, 
                              loss_function='Logloss',
                              eval_metric='AUC',
                              logging_level='Verbose', 
                              metric_period=500)
cat_model.fit(X_train2, y_train2, eval_set=(X_train2, y_train2))

0:	test: 0.8675968	best: 0.8675968 (0)	total: 13.1ms	remaining: 2m 36s
500:	test: 0.9085252	best: 0.9085252 (500)	total: 5.5s	remaining: 2m 6s
1000:	test: 0.9269731	best: 0.9269731 (1000)	total: 11.1s	remaining: 2m 1s
1500:	test: 0.9428407	best: 0.9428407 (1500)	total: 16.8s	remaining: 1m 57s
2000:	test: 0.9568443	best: 0.9568443 (2000)	total: 22.4s	remaining: 1m 51s
2500:	test: 0.9676083	best: 0.9676083 (2500)	total: 27.7s	remaining: 1m 45s
3000:	test: 0.9768102	best: 0.9768102 (3000)	total: 33.2s	remaining: 1m 39s
3500:	test: 0.9836107	best: 0.9836107 (3500)	total: 38.6s	remaining: 1m 33s
4000:	test: 0.9889555	best: 0.9889555 (4000)	total: 44s	remaining: 1m 27s
4500:	test: 0.9929126	best: 0.9929126 (4500)	total: 49.4s	remaining: 1m 22s
5000:	test: 0.9956254	best: 0.9956254 (5000)	total: 55s	remaining: 1m 17s
5500:	test: 0.9973618	best: 0.9973618 (5500)	total: 1m	remaining: 1m 11s
6000:	test: 0.9984829	best: 0.9984829 (6000)	total: 1m 5s	remaining: 1m 5s
6500:	test: 0.9991201	best: 0.

In [30]:
cat_pred = cat_model.predict_proba(X_test)[:, 1]
cat_pred

array([4.86413118e-04, 3.46757569e-02, 1.62288636e-04, ...,
       1.18332417e-04, 3.66459382e-03, 2.62187965e-05])

In [31]:
submission = pd.DataFrame({'id':test['loan_id'], 'isDefault':cat_pred})
submission.to_csv('./submit/baseline_improve.csv', index = None)
submission

,id,isDefault
0,1000575,0.000486
1,1028125,0.034676
2,1010694,0.000162
3,1026712,0.000058
4,1002895,0.000460
...,...,...
4995,1008856,0.317449
4996,1016651,0.000548
4997,1024140,0.000118
4998,1014316,0.003665
